In [18]:
#Helper functions for verification and experimentation
import boto3
import pickle as pkl
import pandas as pd
import requests
import json
import os
import logging

In [27]:
def load_pkl_obj_s3(file_name, bucket):
    s3 = boto3.resource('s3')
    obj = pkl.loads(s3.Bucket(bucket).Object(file_name).get()['Body'].read())
    return obj

def dump_pkl_obj_s3(obj, file_name, bucket):
    s3 = boto3.resource('s3')
    pickle_obj = pkl.dumps(obj)
    response = s3.Object(bucket, file_name).put(Body=pickle_obj)
    return response['ResponseMetadata']

def load_file_s3(object_name, bucket, downloaded_file_name=None):
    s3 = boto3.client('s3')

    if downloaded_file_name is None:
        downloaded_file_name = object_name

    with open(downloaded_file_name, 'wb') as f:
        s3.download_fileobj(bucket, object_name, f)

def dump_file_s3(local_file_name, bucket, target_file_name=None):
    # If S3 object_name was not specified, use file_name
    if target_file_name is None:
        target_file_name = local_file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(local_file_name, bucket, target_file_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

def see_all_files_s3(bucket):
    s3 = boto3.resource('s3')
    s3_bucket = s3.Bucket(bucket)
    files = []
    for obj in s3_bucket.objects.all():
        files.append(obj.key)
    return files
